In [2]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision
import matplotlib.pyplot as plt
import sinabs
import sinabs.activation
import sinabs.layers as sl
from sinabs.from_torch import from_model
import os
import shutil
from sklearn.model_selection import KFold
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)
torch.cuda.empty_cache()

True


Data

In [3]:
class MyData(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.data_path = os.listdir(self.root_dir)
    
    def __getitem__(self, idx):
        data_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, data_name)
        with open(data_item_path, 'rb') as f:
            data = np.load(f)

        # ############## 测试压缩后的性能 ##############
        # data_zeros = np.zeros((100, 350))
        # data_zeros += data[:, ::2] + data[:, 1::2]
        # data = data_zeros
        # #############################################

        data = torch.from_numpy(data).float()
        data = torch.transpose(data, 0, 1)

        if data_name[-6] == '_':
            label = torch.tensor(eval(data_name[-5]), dtype=torch.long)
        else:
            label = torch.tensor(eval(data_name[-6:-4]), dtype=torch.long)
        
        return data, label

    def __len__(self):
        return len(self.data_path)

root_dir_1 = 'F:\Files\PhD/Braille\Data/braille-27letters-sphere/effect-speed/speed-v70-depth2.5/compress-test/compresstime-350/train'
root_dir_2 = 'F:\Files\PhD/Braille\Data/braille-27letters-sphere/effect-speed/speed-v70-depth2.5/compress-test/compresstime-350/test'

train_data = MyData(root_dir_1)
test_data  = MyData(root_dir_2)

In [4]:
print(train_data[0][0].shape)
print(len(train_data), len(test_data))

torch.Size([350, 100])
2160 1080


Training

In [5]:
lr = 1e-4

device = 'cuda:0'
k_folds = 6
kfold = KFold(n_splits=k_folds, shuffle=False)
batch_size = 81
loss_fn = nn.CrossEntropyLoss()

fold_train_acc = []
fold_val_acc = []

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_data)):
    # 模型初始化
    linear_model = nn.Sequential(
        # nn.Linear(100, 200),
        # nn.ReLU(),
        # nn.Linear(200, 27)
        nn.Linear(100, 400),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(400, 600),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(600, 200),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(200, 27)
    )
    linear_model = from_model(linear_model, batch_size=81, input_shape=(1, 350, 100), 
                              add_spiking_output=True).to(device)
    optimizer = torch.optim.Adam(linear_model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    # 分割数据集
    train_sub = Subset(train_data, train_ids)
    val_sub = Subset(train_data, val_ids)
    # 创建数据加载器
    train_loader = DataLoader(train_sub, batch_size, shuffle=True, drop_last=True)
    val_loader  = DataLoader(val_sub, batch_size, drop_last=True)

    # 训练模型
    linear_model.train()
    epochs = 40
    for e in range(epochs):
        running_loss = 0.
        acc = 0
        scheduler.step()
        for i, (input, target) in enumerate(train_loader):
            optimizer.zero_grad()
            linear_model.reset_states()

            input = input.to(device)
            target = target.to(device)
            output = linear_model(input)
            sum_output = output.sum(1)
            loss = loss_fn(sum_output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss

            for j in range(batch_size):
                if sum_output[j].argmax() == target[j]:
                    acc += 1

        print("epoch: %d, accuracy: %.2f%%, running_loss: %.2f, current_lr: %.6f" 
              % (e, acc/len(train_sub)*100, running_loss, scheduler.get_last_lr()[0]) )
    fold_train_acc.append(np.around(acc/len(train_sub)*100, 2))

    # 验证模型
    acc_num = 0
    for i, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            linear_model.reset_states()
            data = data.to(device)
            target = target.to(device)
            output = linear_model(data)
            sum_output = output.sum(1)

        for j in range(batch_size):
            if sum_output[j].argmax() == target[j]:
                acc_num += 1
    print("accuracy on validation set: %.2f%%" % (acc_num/len(val_sub)*100))
    fold_val_acc.append(np.around(acc_num/len(val_sub)*100, 2))

f:\Softwares\Professional\Anaconda3\envs\synsense-gpu\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch: 0, accuracy: 4.28%, running_loss: 72.67, current_lr: 0.000100
epoch: 1, accuracy: 6.39%, running_loss: 71.59, current_lr: 0.000100
epoch: 2, accuracy: 8.72%, running_loss: 71.08, current_lr: 0.000100
epoch: 3, accuracy: 11.89%, running_loss: 68.50, current_lr: 0.000100
epoch: 4, accuracy: 22.00%, running_loss: 60.74, current_lr: 0.000100
epoch: 5, accuracy: 39.17%, running_loss: 44.74, current_lr: 0.000100
epoch: 6, accuracy: 47.33%, running_loss: 35.77, current_lr: 0.000100
epoch: 7, accuracy: 60.00%, running_loss: 26.90, current_lr: 0.000100
epoch: 8, accuracy: 69.39%, running_loss: 20.74, current_lr: 0.000100
epoch: 9, accuracy: 73.22%, running_loss: 17.13, current_lr: 0.000100
epoch: 10, accuracy: 75.67%, running_loss: 15.77, current_lr: 0.000100
epoch: 11, accuracy: 78.61%, running_loss: 12.88, current_lr: 0.000100
epoch: 12, accuracy: 83.39%, running_loss: 10.26, current_lr: 0.000100
epoch: 13, accuracy: 85.00%, running_loss: 9.43, current_lr: 0.000100
epoch: 14, accuracy:

In [6]:
print(fold_train_acc)
print(fold_val_acc)

sum = 0
cnt = 0
for i in fold_train_acc:
    sum += i
    cnt += 1
print(sum/cnt)

sum = 0
cnt = 0
for i in fold_val_acc:
    sum += i
    cnt += 1
print(sum/cnt)

[95.11, 95.89, 95.78, 95.44, 94.33, 95.94]
[86.39, 88.06, 86.67, 88.61, 85.28, 85.0]
95.415
86.66833333333334


In [7]:
model_path = './models/speed70-ct350.pth'
torch.save(linear_model, model_path)

Testing

In [8]:
model = torch.load(model_path)
test_loader  = DataLoader(test_data, batch_size, drop_last=True)

acc_num = 0
for i, (data, target) in enumerate(test_loader):
    with torch.no_grad():
        model.reset_states()
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        sum_output = output.sum(1)

    for j in range(batch_size):
        if sum_output[j].argmax() == target[j]:
            acc_num += 1
print("accuracy on testing set: %.2f%%" % (acc_num/len(test_data)*100))

accuracy on testing set: 94.26%
